# validateDates

on OCR text recognition might not be error free.
you take a glance to transkribus result and all looks good.
nevertheless a sematic check on result is useful.

**our data is a index form a book**
it starts with names (receiver of letter) in maybe multiple lines,
followed by dates, which are sorted, from older to newer.

```text
Friedmann, Ernst
13. 9. 1912
Friese, Carl
13. 2. 1907
20. 2. 1907
Fulda, Ludwig
28. 11. 1898
28. 12. 1898
4. 1. 1899
10. 3. 1890
25. 4. 1899
20. 6. 1900
23. 6. 1900
7. 7. 1900
17. 1. 1901
22. 3. 1901
8. 6. 1901
23. 7. 1904
25. 8. 1904
Glümer, Marie
Sommer 1889
4. 8. 1889
5. 8. 1889
```

with OpenRefine, a json was created

```json
}, {
    "recv": "Fulda, Ludwig",
    "date_when": "1898-12-28",
    "date_text": "28. 12. 1898",
    "uncertain": false
}, {
    "recv": "Fulda, Ludwig",
    "date_when": "1899-01-04",
    "date_text": "4. 1. 1899",
    "uncertain": false,
}, {
    "recv": "Fulda, Ludwig",
    "date_when": "1890-03-10",
    "date_text": "10. 3. 1890",
    "uncertain": false,
}, {
    "recv": "Fulda, Ludwig",
    "date_when": "1899-04-25",
    "date_text": "25. 4. 1899",
    "uncertain": false
}, {
```

only a focused sight at this data, text or refined data, will reveal recognition error
at `10. 3. 1890` (1890-03-10), seen in context, it is supposed to be `10. 3. 1899`;
however we don't want to replace humans, so it's up to a human to decide.

**goal** of this notebook is to find suspicious dates, out of order dates.
if a newer date was present in previous line, one of those two is properly
wrong.

```json
}, {
    "recv": "Fulda, Ludwig",
    "date_when": "1898-12-28",
    "date_text": "28. 12. 1898",
    "uncertain": false
}, {
    "recv": "Fulda, Ludwig",
    "date_when": "1899-01-04",
    "date_text": "4. 1. 1899",
    "uncertain": false,
    "suspicious": "true"
}, {
    "recv": "Fulda, Ludwig",
    "date_when": "1890-03-10",
    "date_text": "10. 3. 1890",
    "uncertain": false,
    "suspicious": "true"
}, {
    "recv": "Fulda, Ludwig",
    "date_when": "1899-04-25",
    "date_text": "25. 4. 1899",
    "uncertain": false
}, {
```


## Released under MIT License

Copyright (c) 2020 Walter Obweger (twitter:`@wobweger`).

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## background

created during [ACDH and CH](https://www.oeaw.ac.at/acdh/) [internship](https://www.oeaw.ac.at/acdh/education/acdh-ch-internships/) 2020

big thanks to the whole ACDH and CH team (twitter: `@ACDH_OeAW`),
special thanks to my instructor [Martin Anton Mueller](https://www.oeaw.ac.at/acdh/team/current-team/martin-anton-mueller/) (twitter:@f46906169)

twitter `#wroACDHitp202003`

In [1]:
import json
import datetime

In [2]:
def validateDates(sFN):
    lDat=[]
    lRecv=[]
    iRecvToClr=0
    zPrev=None
    with open(sFN,'r') as fIn:
        d=json.load(fIn)
        lLetter=d['rows']
        for dLetter in lLetter:
            sRecv=dLetter['recv']
            sSend=dLetter['date_when']
            iUnCer=dLetter['uncertain']
            iIsDate=0
            if len(sSend)==0:
                lPart=sRecv.split(' ')
                for sPart in lPart:
                    try:
                        iVal=int(sPart)
                        iIsDate=1
                    except:
                        pass
            else:
                iIsDate=1
            if iIsDate==0:
                if iRecvToClr>0:
                    lRecv=[]
                    zPrev=None
                    iRecvToClr=0
                lRecv.append(sRecv)
            else:
                dItem={'recv':','.join(lRecv),'date_when':sSend,'date_text':sRecv,'uncertain':iUnCer}
                try:
                    zSend=datetime.datetime.strptime(sSend, '%Y-%m-%d')
                    if zPrev is not None:
                        if zSend<zPrev:
                            lDat[-1]['suspicious']='true'
                            dItem['suspicious']='true'
                            print('properly wrong year recognized')
                except:
                    zSend=None
                lDat.append(dItem)
                if zSend is not None:
                    zPrev=zSend
                iRecvToClr=1
    return lDat

In [3]:
def writeJson(sOutFN,lDat):
    with open(sOutFN,'w') as fOut:
        fOut.write(json.dumps(lDat))

## Arthur Schnitzlers Brief vom 1875 bis 1912

In [8]:
lDat=validateDates('../cmif/ArthurSchnitzler_Briefe_1875_1912_02.json')

properly wrong year recognized
properly wrong year recognized
properly wrong year recognized
properly wrong year recognized
properly wrong year recognized
properly wrong year recognized
properly wrong year recognized


In [9]:
lDat[0]

{'recv': 'Andrian, Leopold,Freiherr von',
 'date_when': '1912-11-18',
 'date_text': '18. 11. 1912',
 'uncertain': False}

In [10]:
writeJson('../ArthurSchnitzler_Briefe_1875_1912_Inhalt_03.json',lDat)

## Arthur Schnitzlers Brief vom 1913 bis 1931

In [4]:
lDat=validateDates('../cmif/ArthurSchnitzler_Briefe_1913_1931_02.json')

properly wrong year recognized
properly wrong year recognized
properly wrong year recognized
properly wrong year recognized
properly wrong year recognized


In [5]:
writeJson('../ArthurSchnitzler_Briefe_1913_1931_Inhalt_03.json',lDat)